# Retrieval-augmented Generation (RAG)

In this example, we will use RAG to generate code based on an external API defined in a Swagger file.

## Dependencies

### LangChain

Here, we will use the LangChain libraries.

In [1]:
import sys
!{sys.executable} -m pip install langchain jq langchain-community langchain-openai

### Model

We will use OpenAI's models, so we need an API key.


In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


## Retrieval

First, we have to load the documents that will be stored in the database.

To this aim, we will read the JSON file describing the API.

In [4]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
import jq

file_path='./opendatahub.json'

with open(file_path) as f:
    data = json.load(f)
    api_url = jq.compile('.servers[0].url').input(data).first()
    print(api_url)

loader = JSONLoader(
         file_path=file_path,
         jq_schema='.paths | to_entries[] | .key as $path | .value | to_entries[] | { path:$path, method:.key, tag:.value.tags[0], summary:.value.summary, parameters: [ {name: .value.parameters[]?.name } ] }',
         text_content=False)

docs = loader.load()

https://tourism.api.opendatahub.com


Checking the number of documents:

In [5]:
len(docs)

152

Let's check the first document to take a look on it.

In [6]:
docs[1]

Document(metadata={'source': '/Users/user/Projects/genai4se-course/opendatahub.json', 'seq_num': 2}, page_content='{"path": "/v1/Accommodation", "method": "post", "tag": "Accommodation", "summary": "POST Insert new Accommodation", "parameters": []}')

Let's use OpenAI's text-embedding-3-large model for creating the embeddings. 

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

For this didactic example, we will use the LangChain's InMemoryVectorStore. According to the [documentation](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.in_memory.InMemoryVectorStore.html), it uses a dictionary and the similarity is calculated using cosine similarity.

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Now let's add the documents to the vector store. In this process, the embeddings are calculated using the defined model.

In [9]:
vector_store.add_documents(documents=docs)

['bdb95d39-ee29-472f-ab10-60dbbc795f13',
 'b6dd174f-7bf2-494e-8cbe-3fe5ba8ce9e8',
 'c37710fe-4063-4d2b-a0a5-e5ab7e84c9cb',
 'c85c8018-47dd-4cb1-9454-90cc2fb15cc7',
 '88aa9c64-ec2d-48c2-beb9-d0f0d8fc4bdf',
 '3647abc8-0cdd-4686-a129-c71deda89389',
 '4aa339f0-3487-4b9a-8d7b-1d3a853de7eb',
 '5ead2d67-1675-4a06-8d7f-6b67cad92c8c',
 'cd861860-08ef-40f5-b4e2-448095e7d769',
 'f20f9137-2ed0-4b36-81b2-c564a764d9f0',
 'ec3841dd-08ee-484a-8c25-10d242faeeb4',
 '551fde91-8a52-498a-87b2-cec94d886a2c',
 '728963d3-dd8b-446e-ae6e-a7a0d0e985cc',
 '28d9dde5-4307-403f-92d1-1720786c3419',
 'c6543bff-f730-46a3-9d65-0dfa679db17a',
 'fc44ec4c-1e67-4673-b170-9b16fa847a7a',
 '4f2193e9-e1dc-47a0-8671-95ea378d35f4',
 '720de197-eca2-4d5b-a5ea-c1521eefe855',
 '737ca9ef-4442-4212-8a9b-e1ec13ed4455',
 'a57e12fc-3a73-41f3-9360-a702ed746cee',
 '6706f4d1-4d91-4e55-ac64-a663ef3d7d42',
 '654151e7-0af3-4425-bd50-533e56a77d81',
 'a4e540a6-53a1-46ce-b426-110bf04ff1c0',
 '98e7a656-cf0e-47a6-96e1-630dc27b176f',
 '88293acf-da0a-

Let's create a function to retrieve the relevant documents given the text definition of a task.

In [10]:
def retrieve(task):
    retrieved_docs = vector_store.similarity_search(task)
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

Let's check if it works:

In [11]:
task = "Write a piece of code to list the events in the last year."
retrieve(task)

'{"path": "/v1/EventShort", "method": "get", "tag": "EventShort", "summary": "GET EventShort List", "parameters": [{"name": "pagenumber"}, {"name": "pagesize"}, {"name": "startdate"}, {"name": "enddate"}, {"name": "datetimeformat"}, {"name": "source"}, {"name": "eventlocation"}, {"name": "onlyactive"}, {"name": "websiteactive"}, {"name": "communityactive"}, {"name": "active"}, {"name": "eventids"}, {"name": "webaddress"}, {"name": "sortorder"}, {"name": "seed"}, {"name": "language"}, {"name": "langfilter"}, {"name": "optimizedates"}, {"name": "latitude"}, {"name": "longitude"}, {"name": "radius"}, {"name": "polygon"}, {"name": "fields"}, {"name": "lastchange"}, {"name": "publishedon"}, {"name": "searchfilter"}, {"name": "rawfilter"}, {"name": "rawsort"}, {"name": "removenullvalues"}]}\n\n{"path": "/v1/Event", "method": "get", "tag": "Event", "summary": "GET Event List", "parameters": [{"name": "language"}, {"name": "pagenumber"}, {"name": "pagesize"}, {"name": "idlist"}, {"name": "locf

## Generation

Let's use OpenAI's GPT-4o-mini to generate the answer.

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini")

Here we define the prompt template, leaving the space for information retrieved about the API and the task. We also add the information about the API url so the generated code will be runnable. Let's save the result in a Python file.

In [18]:
def generate(task, api_info, api_url):

    prompt_template = ChatPromptTemplate([
        ("system", "You are a developer using an API to implement a solution in Python."),
        ("user", "Below, there is the information about the API you need to use {apiInfo}. Your task is: {task}. Just return the code without anything else. The API URL is {apiUrl}")
    ]) 
    
    prompt = prompt_template.invoke({"apiInfo": api_info, "task": task, "apiUrl": api_url})
    answer = llm.invoke(prompt)
    result = answer.content
    
    answer.pretty_print()
    
    if "```python" in result:
            result = result[10:-3]
    
    output_path = "output/main.py"
    with open(output_path, "w") as f:
        f.write(result)

In [19]:
api_info = retrieve(task)
generate(task, api_info, api_url)

================================== Ai Message ==================================

```python
import requests
from datetime import datetime, timedelta

# Define the API URL
api_url = "https://tourism.api.opendatahub.com/v1/EventShort"

# Calculate the date range for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Format the dates
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Define the parameters
params = {
    "startdate": start_date_str,
    "enddate": end_date_str,
    "onlyactive": "true",
    "pagesize": 100,
    "pagenumber": 1
}

# Make the GET request
response = requests.get(api_url, params=params)

# Check the response status
if response.status_code == 200:
    events = response.json()
    print(events)
else:
    print(f"Error: {response.status_code}, {response.text}")
```
